# IBM Ponder This - April 2020 - Challenge

by Walter Sebastian Gisler

## Problem Statement

The original problem statement can be found here: https://www.research.ibm.com/haifa/ponderthis/challenges/April2020.html

This month's challenge is inspired by the COVID-19 pandemic.

Suppose that the world has five cities and they are all connected to one another as depicted by the five-verticed graph in the following picture:

![Example Graph](ponder0420.jpg "Example Graph")

Let's assume that the infection passes daily along the edges. So, if on day t, "D" is infected and "C" is healthy, then "C" has a 10% chance of getting infected, by "D", on day t+1.

If "A" is infected at time 0, after ten days, there is about a 29.16521896% probability that all five will be infected.

Find a graph with no more than eight vertices that gives a probability of 70.00% (accurate to the second decimal digit after the decimal point) of all cities/vertices being infected after ten days.

Give your answer as an adjacency matrix.
For example, the adjacency matrix of the graph depicted above is

01110 
10001 
10010 
10101 
01010

Bonus '*' for getting the closest to 70%.

Update (3/4): We meant to ask after 19 days (which can give solution with a better approximation), but we will accept also answers for 10 days.

# Solution

Interesting problem! It sounds really simple, but actually it is more difficult than expected. This is a kind of problem where one is easily tempted to calculate the probabilities wrongly, so even getting the same result as in the example is not completely trivial. A great opportunity to refresh some topics from statistics: *Markov chains, probability of dependent and independent events, combinatorics, graph theory...* A nice way of spending a Sunday afternoon.

In the February 2020 Challenge (http://www.research.ibm.com/haifa/ponderthis/challenges/February2020.html) I used Markov Chains to solve the problem. This months challenge can be approached with the same toolset.

I always like to start by making sure I understand the example, so I will first try to verify that the probability in the example is indeed 29.16521896%

Given the example graph above with the vertices ${A, B, C, D, E}$ and the edges $e = {AB, AC, AD, BE, ED,CD}$, we have the following states:

${A, B, C, D, E, AB, AC, AD, AE, BC, BD, BE, CD, CE, DE, ABC, ABD, ABE, ACD, ACE, BCD, BCE, BDE, CDE, ABCD, ABCE, ACDE, ABCDE}$

The states can be interpreted as follows: ABC means that vertice A, B and C are infected, and the other vertices (D and E) are not.

Every possible situation after any number of days can be described using one of these states. For completeness we should also include a state where no vertice is infected, but we can leave this away, since we know that A is always infected to start.

If we know the probability of moving from one state to another, we can easily calculate the probability for every single one of these states after any number of days. Given the transition matrix $M$, and the initial vector $V = {1, 0, 0, ... , 0, 0}$ which says that the first vertice is infected on day 0, the probability for each state after $x$ days can be calculated as follows:

$V*M^x$

Easy! So, all we need to do is to calculate the values in the transition matrix for the given graph.

It is probably easiest, if we start with a simple example. What is the probability of moving from the state ABC to the state ABCD in a one day period?

This can also be stated as the probability p1 of D getting infected in a given time period and the probability p2 of E not getting infected in the same time period.

D has an edge with two infected vertices: A and C. This means that there are three ways that D can get infected:

- D can get infected by A, but not by C
- D can get infected by C, but not by A
- D can get infected by A and C

The probability of the first even is $0.1*0.9$ - 10% chance of catching the virus from A and 90% chance of not catching it from C. The same applies to the second option.

The third option has a probability of $0.1*0.1$.

Combined, the probability of D getting infected, when A, B and C are infected on the previous day is $0.1*0.9 + 0.1*0.9 + 0.1*0.1 = 0.19$

E can get the virus through one edge only, so the probability of not catching it is $1-0.1 = 0.9$

The combined probability of D getting infected and E not getting infected is therefore $0.19*0.9 = 17.1%$

Let's formalize this and implement a method to calculate the probability of a vertice getting the virus, given a list of infected vertices, a list of edges and an infection rate r. I am taking a shortcut here, but essentially, I am creating a list of edges that connect v with an infected vertice. Given $x$ infected vertices, and an infection rate of $r$, I can write the probability of $v$ getting infected in a period as follows:

$\sum_{\substack{1<i<x}} r^i (1-r)^{x-i} {x\choose i} $

This can be written without the sum symbol and without the binominal coefficient. I am using wolframalpha to simplify it:

https://www.wolframalpha.com/input/?i=sum%28+r%5Ei+*+%281-r%29%5E%28x-i%29+*ncr%28x%2Ci%29+%2C+i+%3D+1%2C+x%29

The result is as follows:

$\left(\left(\frac{1}{1-r}\right)^x - 1\right) (1-r)^x $

In [1]:
def infection_probability(v, infected_vertices, edges, r):
    # probability that v gets effected if all infected_vertices are infected and the vertices are connected by edges and the probability of getting infected by one single neighboring vertice is r
    prob = 0
    dangerous_edges = 0
    for infected in infected_vertices:
        if (infected, v) in edges or (v, infected) in edges:
            dangerous_edges += 1
    return ((1/(1-r))**dangerous_edges-1)*(1-r)**dangerous_edges

We can verify that this is working correctly by doing the same calculation that we did manually above:

In [2]:
p = infection_probability('D', ['A','B','C'], [('A','B'), ('A','C'), ('A','D'), ('B','E'), ('E','D'),('C','D')], 0.1)
print(p)

0.19000000000000014


Using this method, we can now also define a method to calculate the probability of moving from one state to another in a given time period. To do this, we first check if all vertices in $s1$ are included in the target state $s2$. If this is not the case, the transition probability is 0. Otherwise, we identify all vertices in $s2$ that are not in $s1$. These are the vertices that are at risk of getting infected. All remaining vertices in the graph are not at risk for getting infected. The transition probability is the product of the combined probability of all vertices that are at risk getting infected, and all other vertices not getting infected.

In [3]:
def transition_probability(s1, s2, vertices, edges, r):
    s1 = set(s1)
    s2 = set(s2)
    if len(s1.intersection(s2)) < len(s1):
        return 0
    else:
        to_infect = s2 - s1
        prod = 1
        for v in to_infect:
            prod = prod*infection_probability(v, s1, edges, r)
        for v in vertices:
            if v not in to_infect and v not in s1:
                prod = prod*(1-infection_probability(v, s1, edges, r))
        return prod

Again, let's test this by using the above example:

In [4]:
p = transition_probability(['A','B','C'], ['A','B','C','D'], ['A','B','C','D','E'], [('A','B'), ('A','C'), ('A','D'), ('B','E'), ('E','D'),('C','D')], 0.1)
print(p)

0.1710000000000001


Perfect! We have all the components working to calculate the transition matrix and to calculate the probabilities of each state after a given number of days. The following method is constructing the transition matrix $m$ and then multiplying the start vector $sv$ with $m^{tt}$. $tt$ is the number of periods.

In [5]:
from itertools import combinations
import numpy as np

def calculate_with_markov(vertices, edges, tt, r):
    states = []
    for num_nodes in range(1,len(vertices)+1):
        states += list(combinations(vertices, num_nodes))
    tm = [[0 for st in states] for st in states]
    for i1,s1 in enumerate(states):
        for i2,s2 in enumerate(states):
            tm[i1][i2] = transition_probability(s1, s2, vertices, edges, r)
    m = np.matrix(tm)
    sv = np.matrix([1 if state in [('A', ), (0, )] else 0 for state in states])
    return (sv*m**tt)[0,-1]

sv is a vector that gives the probability of being in a certain state after tt time periods. We are only interested in the probability of ending up in the last state (which means, everyone is infected), so we are returing the last element of this vector.

Let's test if this is working:

In [6]:
vertices = ['A','B','C','D','E']
edges = {('A','B'),('A','C'),('A','D'),('B','E'),('E','D'),('C','D')}
tt = 10
r = 0.1

p = calculate_with_markov(vertices, edges, tt, r)
print(p)

0.29165218960150663


Wonderful! We are getting the correct percentage. This means that we now have a method that allows us to calculate the probability of all vertices being infected for any given graph, after any given number of days. Hence, we could now use brute-force and try all possible graphs with up to 8 vertices to find the one that gets closest to 70%. A method that generates all possible graphs and finds the graph with the best percentage looks as follows:

In [7]:
from time import time

def find_graph_with_probability(vertices, target_probability, num_phases, infection_rate):
    start_time = time()
    count = 0
    all_edges = list(combinations(vertices, 2))
    best_diff = 10000
    best_graph = {}
    for selected_num_edges in range(len(vertices)-1, len(all_edges)+1): # we look at all connected graphs. A condition for a graph to be connected is that the number of edges is at least |vertices|-1
        for edges in combinations(all_edges, selected_num_edges):
            count += 1
            diff = abs(calculate_with_markov(vertices, edges, num_phases, infection_rate)-target_probability)
            if diff < best_diff:
                best_diff = diff
                best_graph = edges
    print(best_graph)
    print(calculate_with_markov(vertices, best_graph, num_phases, infection_rate))
    end_time = time()
    print('Number of graphs evaluated: %i, time spent total: %f, graphs per second: %f'%(count, end_time-start_time, count/(end_time-start_time)))

Note, that we are only generating graphs with at least |vertices|-1 edges. Graphs with fewer edges are not connected, which means that in such a graph it is never possible that all vertices are infected. This reduces the number of graphs a bit, but the reduction is relatively modest.

Let's try this with all graphs with 5 vertices:

In [8]:
vertices = ['A','B','C','D','E']
find_graph_with_probability(vertices, 0.7, 10, 0.1)

(('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('C', 'D'), ('C', 'E'), ('D', 'E'))
0.7031035050481966
Number of graphs evaluated: 848, time spent total: 1.999892, graphs per second: 424.022898


After about 2 seconds we find a graph with a percentage of 70.31%. Before we continue with graphs with a higher number of vertices, let's do some math first. How many graphs will we be evaluating if we consider graphs with 6, 7 or 8 vertices?

The number is as follows:

5 vertices:
sum(ncr(ncr(5,2),i), i = 5-1 to ncr(5,2)) = 848
https://www.wolframalpha.com/input/?i=sum%28ncr%28ncr%285%2C2%29%2Ci%29%2C+i+%3D+5-1+to+ncr%285%2C2%29%29

6 vertices:
sum(ncr(ncr(6,2),i), i = 6-1 to ncr(6,2)) = 30'827
https://www.wolframalpha.com/input/?i=sum%28ncr%28ncr%286%2C2%29%2Ci%29%2C+i+%3D+6-1+to+ncr%286%2C2%29%29

7 vertices
sum(ncr(ncr(7,2),i), i = 7-1 to ncr(7,2)) = 2'069'256
https://www.wolframalpha.com/input/?i=sum%28ncr%28ncr%287%2C2%29%2Ci%29%2C+i+%3D+7-1+to+ncr%287%2C2%29%29

8 vertices:
sum(ncr(ncr(8,2),i), i = 8-1 to ncr(8,2)) = 267'936'278
https://www.wolframalpha.com/input/?i=sum%28ncr%28ncr%288%2C2%29%2Ci%29%2C+i+%3D+8-1+to+ncr%288%2C2%29%29

So, if we want to test all graphs with 8 vertices, we will be looking at about 300'000 times the number of graphs as with 5 vertices. I expect the time to evaluate a graph to be higher if we have a graph with more vertices, but for simplicity we can assume that the amount of time that is needed is constant. Even then, we would be looking at about 170 hours of computing on my computer. In reality, this number would be much higher due to the increasing effort to multiple the matrices.

Sure, we can throw more hardware at this, but it seems like an overkill. There must be better ways....

## A reduced number of graphs

When facing a large problem in optimization, we normally want to reduce the search space. Are there any solutions that can be ignored? Instead of doing the full evaluation, can we cut off part of the solution space?

Sure, there are a couple of things. First, if a graph is not connected (which is not guaranteed only by making sure that the number of edges is $>= |vertices|-1$), we can ignore it. It is certainly less costly to check for connectivity than to evaluate the graph.

An even bigger reduction can be achieved, if we remove graphs that are equivalent. For example, the two graphs that are defined by their edges as follows

$G1 = \{AB, BC\}$

$G2 = \{AC, BC\}$

can be considered identical. How? Well, if we don't label the vertices, these two graphs look exactly the same. In particular here, if you swap the $B$ and $C$ label in $G1$, you get $G2$.

Obviously, this second method can reduce the number of graphs significantly.

The following paper explains how to efficiently generate graphs without having any equivalent graphs: B. D. McKay and A. Piperno, Practical Graph Isomorphism, II, J. Symbolic Computation (2013) 60 94-112. http://dx.doi.org/10.1016/j.jsc.2013.09.003. The authros have also published and open-sourced a tool called "Nauty" that can be used to generate graphs like this: http://users.cecs.anu.edu.au/~bdm/nauty/

The follwing command generates all unique, undirected, unlabeled graphs with 5 vertices and saves them to a text file

./geng 5 -c > g5.txt

The tool is blazing fast. For 8 vertices, it just takes some seconds, and generates a bit more than 11000 graphs. Much more doable than to evaluate 267 Mio graphs.

So, instead of looking at all graphs, we can concentrate on a much smaller subset of graphs:

In [9]:
import networkx as nx

def find_graph_with_probability_in_file(file_path, target_probability, num_phases, infection_rate):
    graphs = nx.read_graph6(file_path)
    best_diff = 10000
    best_graph = None
    for G in graphs:
        vertices = G.nodes()
        edges = G.edges()
        for v in vertices:
            edges_copied = [[e[0], e[1]] for e in edges]
            for index, edge in enumerate(edges_copied):
                if edge[0] == v:
                    edge[0] = -1
                if edge[1] == v:
                    edge[1] = -1
                if edge[0] == 0:
                    edge[0] = v
                if edge[1] == 0:
                    edge[1] = v
                if edge[0] == -1:
                    edge[0] = 0
                if edge[1] == -1:
                    edge[1] = 0
                edges_copied[index] = (edge[0], edge[1])
            diff = abs(calculate_with_markov(vertices, edges_copied, num_phases, infection_rate) - target_probability)
            if diff < best_diff:
                best_diff = diff
                best_graph = edges_copied
    print(best_graph)
    for row in vertices:
        line = ''
        for column in vertices:
            line += '1' if (row,column) in best_graph or (column, row) in best_graph else '0'
        print(line)
    print(calculate_with_markov(vertices, best_graph, num_phases, infection_rate))

Note, that we are using the networkx package to import the graphs that were generated with the "Nauty" tool. Also, we need to look at a graph in different ways. Every node could potentially be the start "A" node, so for each graph that Nauty generated, we need to try all variations of start nodes. Let's run this for 5 and 6 node graphs:

In [10]:
find_graph_with_probability_in_file('g5.txt', 0.7, 10, 0.1)

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]
01111
10111
11011
11101
11110
0.7031035050481966


In [11]:
find_graph_with_probability_in_file('g6.txt', 0.7, 10, 0.1)

[(4, 2), (4, 3), (4, 0), (4, 5), (1, 2), (1, 3), (1, 0), (1, 5), (2, 0), (2, 5), (3, 0), (3, 5), (0, 5)]
011111
101101
110011
110011
101101
111110
0.6768401632646808


This is signicantly faster than the brute force approach. I won't run this for 8 vertice graphs in the Jupyter notebook since it is taking a lot of time, but here are solutions for the 10 and 19 day scenario:

The graph that is given by the set of edges E = {(4, 2), (4, 3), (4, 0), (4, 5), (4, 6), (4, 7), (1, 3), (1, 0), (1, 5), (1, 6), (1, 7), (2, 0), (2, 5), (2, 6), (2, 7), (3, 0), (3, 5), (3, 6), (3, 7), (0, 6), (0, 7), (5, 7)} has an expected percentage of infected vertices of 70.00748730017623% after 10 days. Let's verify.

In [12]:
v = [0,1,2,3,4,5,6,7]
e = [(4, 2), (4, 3), (4, 0), (4, 5), (4, 6), (4, 7), (1, 3), (1, 0), (1, 5), (1, 6), (1, 7), (2, 0), (2, 5), (2, 6), (2, 7), (3, 0), (3, 5), (3, 6), (3, 7), (0, 6), (0, 7), (5, 7)]
p = calculate_with_markov(v, e, 10, 0.1)
print(p)

0.7000748730017624


A graph that has an expected percentage of infected vertices of 70.00008653156908% after 19 days is the graph given by the set of edges {(5, 3), (5, 0), (1, 4), (1, 0), (1, 6), (2, 0), (2, 6), (2, 7), (3, 6), (3, 7), (4, 7), (0, 7)}:

In [13]:
v = [0,1,2,3,4,5,6,7]
e = [(5, 3), (5, 0), (1, 4), (1, 0), (1, 6), (2, 0), (2, 6), (2, 7), (3, 6), (3, 7), (4, 7), (0, 7)]
p = calculate_with_markov(v, e, 19, 0.1)
print(p)

0.7000008653156906


## Making it even faster...

Ok, at this point we are done, but there is one last trick to make the process even faster. In our initial example we had several states that were not needed. For example, $BE$ was not needed, because it doesn't include $A$. $A$ is always infected, so it always has to be part of a state. Similarly, $AE$ is not a state that we will ever encounter, because $A$ can never infect $E$ directly. This means, that we could have reduced the number of states in our transition matrix significantly. The matrix multiplication is obviously the most time intense step in this calculation, so let's remove all states from the transition matrix that can't be found and then compare the speed.

In [14]:
def calculate_faster_with_markov(vertices, edges, tt, r):
    states = []
    for num_nodes in range(1,len(vertices)+1):
        states += list(combinations(vertices, num_nodes))
    states = [state for state in states if 0 in state or 'A' in state] # let's remove all states that don't contain the initial state
    tm = [[0 for st in states] for st in states]
    for i1,s1 in enumerate(states):
        for i2,s2 in enumerate(states):
            tm[i1][i2] = transition_probability(s1, s2, vertices, edges, r)
    m = np.matrix(tm)
    sv = np.matrix([1 if state in [('A', ), (0, )] else 0 for state in states])
    return (sv*m**tt)[0,-1]

So, in this example, I just removed states that don't include the start node. Let's test how much faster this makes the probability calculation:

In [15]:
st = time()
print(calculate_with_markov(v,e, 19, 0.1))
et = time()
print(et-st)

st = time()
print(calculate_faster_with_markov(v,e, 19, 0.1))
et = time()
print(et-st)

0.7000008653156906
0.1711578369140625
0.7000008653156906
0.052659034729003906


This simple improvement speeds up the calculation by about factor 4. Let's now also remove states that can't be reached. For this purpose, we can use the networkx package to test if a graph is connected or not.

In [16]:
def connected(vertices, edges):
    if len(vertices) == 1:
        return True
    G = nx.Graph()
    for v in vertices:
        G.add_node(v)
    count = 0
    for edge in edges:
        if edge[0] in vertices and edge[1] in vertices:
            G.add_edge(edge[0], edge[1])
            count += 1
    return count > 0 and nx.is_connected(G)

In [17]:
def calculate_even_faster_with_markov(vertices, edges, tt, r):
    states = []
    for num_nodes in range(1,len(vertices)+1):
        states += list(combinations(vertices, num_nodes))
    states = [state for state in states if (0 in state or 'A' in state) and connected(state, edges)] # let's remove all states that don't contain the initial state
    tm = [[0 for st in states] for st in states]
    for i1,s1 in enumerate(states):
        for i2,s2 in enumerate(states):
            tm[i1][i2] = transition_probability(s1, s2, vertices, edges, r)
    m = np.matrix(tm)
    sv = np.matrix([1 if state in [('A', ), (0, )] else 0 for state in states])
    return (sv*m**tt)[0,-1]

In [18]:
def find_graph_with_probability_in_file_faster(file_path, target_probability, num_phases, infection_rate, calculation_method):
    graphs = nx.read_graph6(file_path)
    best_diff = 10000
    best_graph = None
    for G in graphs:
        vertices = G.nodes()
        edges = G.edges()
        for v in vertices:
            edges_copied = [[e[0], e[1]] for e in edges]
            for index, edge in enumerate(edges_copied):
                if edge[0] == v:
                    edge[0] = -1
                if edge[1] == v:
                    edge[1] = -1
                if edge[0] == 0:
                    edge[0] = v
                if edge[1] == 0:
                    edge[1] = v
                if edge[0] == -1:
                    edge[0] = 0
                if edge[1] == -1:
                    edge[1] = 0
                edges_copied[index] = (edge[0], edge[1])
            diff = abs(calculation_method(vertices, edges_copied, num_phases, infection_rate) - target_probability)
            if diff < best_diff:
                best_diff = diff
                best_graph = edges_copied
    print(best_graph)
    for row in vertices:
        line = ''
        for column in vertices:
            line += '1' if (row,column) in best_graph or (column, row) in best_graph else '0'
        print(line)
    print(calculation_method(vertices, best_graph, num_phases, infection_rate))

In [19]:
print('Our original approach:')
st = time()
find_graph_with_probability_in_file_faster('g6.txt', 0.7, 10, 0.1, calculate_with_markov)
et = time()
print('%f seconds'%(et-st))

print('Speed up by removing states that do not contain the initial state:')
st = time()
find_graph_with_probability_in_file_faster('g6.txt', 0.7, 10, 0.1, calculate_faster_with_markov)
et = time()
print('%f seconds'%(et-st))

print('In addition, also remove states that can not be reached:')
st = time()
find_graph_with_probability_in_file_faster('g6.txt', 0.7, 10, 0.1, calculate_even_faster_with_markov)
et = time()
print('%f seconds'%(et-st))

Our original approach:
[(4, 2), (4, 3), (4, 0), (4, 5), (1, 2), (1, 3), (1, 0), (1, 5), (2, 0), (2, 5), (3, 0), (3, 5), (0, 5)]
011111
101101
110011
110011
101101
111110
0.6768401632646808
7.386820 seconds
Speed up by removing states that do not contain the initial state:
[(4, 2), (4, 3), (4, 0), (4, 5), (1, 2), (1, 3), (1, 0), (1, 5), (2, 0), (2, 5), (3, 0), (3, 5), (0, 5)]
011111
101101
110011
110011
101101
111110
0.6768401632646808
2.662439 seconds
In addition, also remove states that can not be reached:
[(4, 2), (4, 3), (4, 0), (4, 5), (1, 2), (1, 3), (1, 0), (1, 5), (2, 0), (2, 5), (3, 0), (3, 5), (0, 5)]
011111
101101
110011
110011
101101
111110
0.6768401632646808
2.686991 seconds


As we can observe, the last approach, with invalid states removed from the transition matrix is even faster. In our test with all graphs with 6 vertices, it is about 3 times faster. We would expect the performance gain to be even higher with a higher number of vertices. Let's therefore try to find the optimal solution for 8 vertice graphs and time it:

In [20]:
start_time = time()
find_graph_with_probability_in_file_faster('g8.txt', 0.7, 10, 0.1, calculate_even_faster_with_markov)
end_time = time()
print('Used %f seconds'%(end_time-start_time))

[(4, 2), (4, 3), (4, 0), (4, 5), (4, 6), (4, 7), (1, 3), (1, 0), (1, 5), (1, 6), (1, 7), (2, 0), (2, 5), (2, 6), (2, 7), (3, 0), (3, 5), (3, 6), (3, 7), (0, 6), (0, 7), (5, 7)]
01111011
10010111
10001111
11001111
10110111
01111001
11111000
11111100
0.7000748730017624
Used 3153.824736 seconds


Ok, just short of an hour. Not great, but actually really good compared to our initial very conservative estimate of 166 hours. I'm sure we could speed it up even more if we write the code in a different language and/ or run it on better hardware.